In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
# loop from page 1 to page 34 (Date 24 Jan 2024), page url examlpes: 'https://visitseattle.org/events/page/1'

pages = range(1,35)
events = []
for page in pages:
    url = 'https://visitseattle.org/events/page/' + str(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    selector = 'div.search-result-preview > div > h3 > a'
    a_eles=soup.select(selector)
    events = events + [x['href'] for x in a_eles]

events

['https://visitseattle.org/events/aryana-leon/',
 'https://visitseattle.org/events/colorworks/',
 'https://visitseattle.org/events/dar-williams/',
 'https://visitseattle.org/events/deb-perelman/',
 'https://visitseattle.org/events/emanuel-brown/',
 'https://visitseattle.org/events/james-miles/',
 'https://visitseattle.org/events/madeline-pendleton/',
 'https://visitseattle.org/events/october-london/',
 'https://visitseattle.org/events/tacoma-stars-vs-empire-strykers/',
 'https://visitseattle.org/events/dungeons-and-drag-queens/',
 'https://visitseattle.org/events/global-rhythms-yungchen-lhamo/',
 'https://visitseattle.org/events/harlem-globetrotters-2024-world-tour/',
 'https://visitseattle.org/events/seattle-kraken-vs-st-louis-blues/',
 'https://visitseattle.org/events/shelf-nunny/',
 'https://visitseattle.org/events/super-diamond-the-neil-diamond-tribute-band/',
 'https://visitseattle.org/events/the-disorderlies-2/',
 'https://visitseattle.org/events/compania-nacional-de-danza/',
 'h

In [19]:
eventdata = []

for event in events:
    res = requests.get(event)

    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        
        name = soup.select_one('div.medium-6.columns.event-top > h1')
        date_time = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)')
        location = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)')
        event_type = soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)')
        region = soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)')

        eventdata.append({
            "Name": name.get_text(strip=True) if name else "Not found",
            "Date & Time": date_time.get_text(strip=True) if date_time else "Not found",
            "Location": location.get_text(strip=True) if location else "Not found",
            "Type": event_type.get_text(strip=True) if event_type else "Not found",
            "Region": region.get_text(strip=True) if region else "Not found"
        })

        # eventdata.append({
        #     "Name": name,
        #     "Date & Time": date_time,
        #     "Location": location,
        #     "Type": event_type,
        #     "Region": region
        # })


df = pd.DataFrame(eventdata)


df.to_csv("events.csv", index=False)